## Most Popular

### Configuración Inicial

In [1]:
!pip uninstall -y numpy
!pip install numpy==1.26

Found existing installation: numpy 2.2.6
Uninstalling numpy-2.2.6:
  Successfully uninstalled numpy-2.2.6
  Obtaining dependency information for numpy==1.26 from https://files.pythonhosted.org/packages/d2/2f/b42860931c1479714201495ffe47d74460a916ae426a21fc9b68c5e329aa/numpy-1.26.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Using cached numpy-1.26.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (53 kB)
Using cached numpy-1.26.0-cp311-cp311-macosx_10_9_x86_64.whl (20.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
cornac 2.3.3 requires numpy>2.0.0, but you have numpy 1.26.0 which is incompatible.


In [2]:
!pip install scikit-surprise --no-build-isolation --no-deps
!pip install memory_profiler

### Instalación de Librerías

In [3]:
import pandas as pd
import gdown
import time
from memory_profiler import memory_usage
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations
import numpy as np

### Importación de los Datos

In [4]:
gdown.download(id='1eGDDR1wlvR99eoCZG2owChy2dhkPp4yx', output='training_ratings.csv', quiet=False)
gdown.download(id='1oHo9HLB6SzeqZs76FCkfQ1irSQepqp16', output='validation_ratings.csv', quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1eGDDR1wlvR99eoCZG2owChy2dhkPp4yx
From (redirected): https://drive.google.com/uc?id=1eGDDR1wlvR99eoCZG2owChy2dhkPp4yx&confirm=t&uuid=c07399ba-841a-4868-bbce-14168d43e533
To: /Users/mzambran/Downloads/training_ratings.csv
100%|██████████| 205M/205M [02:10<00:00, 1.58MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1oHo9HLB6SzeqZs76FCkfQ1irSQepqp16
To: /Users/mzambran/Downloads/validation_ratings.csv
100%|██████████| 64.4M/64.4M [00:17<00:00, 3.73MB/s]


'validation_ratings.csv'

In [5]:
df_train = pd.read_csv('training_ratings.csv')
df_val = pd.read_csv('validation_ratings.csv')

In [6]:
# dataset mechanics
gdown.download(id='1cVGSLNVqxrAoKzeqxt_FfQ4Ggs9VvCDO', output='mechanics.csv', quiet=False)
df_mechanics = pd.read_csv('mechanics.csv')

Downloading...
From: https://drive.google.com/uc?id=1cVGSLNVqxrAoKzeqxt_FfQ4Ggs9VvCDO
To: /Users/mzambran/Downloads/mechanics.csv
100%|██████████| 7.05M/7.05M [00:03<00:00, 1.92MB/s]


### Preprocesamiento de Datos

In [7]:
df_mechanics.set_index('BGGId', inplace=True)
print("Datos de mecánicas cargados y listos.")

# --- Calcular la popularidad de los ítems ---
# Usamos el dataframe de entrenamiento COMPLETO (df_train)
item_popularity = df_train['item'].value_counts().to_dict()
total_interactions = len(df_train)

# Convertimos las cuentas en probabilidades para el cálculo de novedad
item_popularity_prob = {item_id: count / total_interactions for item_id, count in item_popularity.items()}
print(f"Popularidad calculada para {len(item_popularity)} ítems.")

Datos de mecánicas cargados y listos.
Popularidad calculada para 16748 ítems.


In [8]:
def novelty_at_k(group, k, popularity_prob):
    """Calcula la Novedad@K para un solo usuario/grupo."""
    group = group.sort_values('score', ascending=False)
    topk_items = group.head(k)['itemID']

    novelty_scores = []
    for item_id in topk_items:
        prob = popularity_prob.get(item_id, 1e-6)
        novelty_scores.append(-np.log2(prob))

    return np.mean(novelty_scores) if novelty_scores else 0.0

def diversity_at_k(group, k, mechanics_df):
    """Calcula la Diversidad@K (Intra-List Diversity) para un solo usuario/grupo."""
    group = group.sort_values('score', ascending=False)
    topk_items = group.head(k)['itemID'].tolist()

    topk_items = [item for item in topk_items if item in mechanics_df.index]

    if len(topk_items) < 2:
        return 0.0

    item_vectors = mechanics_df.loc[topk_items].values

    dissimilarity_sum = 0
    num_pairs = 0
    for i in range(len(item_vectors)):
        for j in range(i + 1, len(item_vectors)):
            sim = cosine_similarity([item_vectors[i]], [item_vectors[j]])[0][0]
            dissimilarity_sum += (1 - sim)
            num_pairs += 1

    return dissimilarity_sum / num_pairs if num_pairs > 0 else 0.0

In [9]:
df_train.drop_duplicates(inplace=True, subset=['user', 'item'])
df_val.drop_duplicates(inplace=True, subset=['user', 'item'])

In [10]:
item_avg_ratings = df_train.groupby('item')['rating'].mean()

item_avg_ratings_dict = item_avg_ratings.to_dict()

global_avg_rating = df_train['rating'].mean()
print(f"Se calculó el promedio para {len(item_avg_ratings)} ítems.")
print(f"El rating promedio global es: {global_avg_rating:.2f}")

Se calculó el promedio para 16748 ítems.
El rating promedio global es: 7.15


In [11]:
predictions = df_val['item'].map(item_avg_ratings_dict).fillna(global_avg_rating)

In [12]:
mse = mean_squared_error(df_val['rating'], predictions)
mae = mean_absolute_error(df_val['rating'], predictions)
rmse = mse ** (1/2)
print(f"\nEl Error Cuadrático Medio (RMSE) del modelo Most Popular es: {rmse:.4f}")
print(f"El Error Absoluto Medio (MAE) del modelo Most Popular es: {mae:.4f}")


El Error Cuadrático Medio (RMSE) del modelo Most Popular es: 1.3132
El Error Absoluto Medio (MAE) del modelo Most Popular es: 0.9943


In [13]:
def evaluar_most_popular_rmse(df_train, df_val):
    """
    Entrena, predice y evalúa el modelo Most Popular, retornando el RMSE.
    """
    # Entrenamiento
    item_avg_ratings = df_train.groupby('item')['rating'].mean().to_dict()
    global_avg_rating = df_train['rating'].mean()

    # Predicción
    predictions = df_val['item'].map(item_avg_ratings).fillna(global_avg_rating)

    # Evaluación
    mse = mean_squared_error(df_val['rating'], predictions)
    mae = mean_absolute_error(df_val['rating'], predictions)
    rmse = mse ** (1/2)

    return rmse, mae

start_time = time.time()
rmse_resultado, mae_resultado = evaluar_most_popular_rmse(df_train, df_val)
end_time = time.time()
elapsed_time = end_time - start_time

memoria_usada = memory_usage((evaluar_most_popular_rmse, (df_train, df_val)))
memoria_max = max(memoria_usada) - min(memoria_usada)

print(f"RMSE: {rmse_resultado:.4f}")
print(f"MAE: {mae_resultado:.4f}")
print(f"Tiempo de ejecución: {elapsed_time:.2f} segundos")
print(f"Memoria utilizada (pico): {memoria_max:.2f} MB")

RMSE: 1.3132
MAE: 0.9943
Tiempo de ejecución: 0.67 segundos
Memoria utilizada (pico): 179.40 MB


In [14]:
from sklearn.metrics import ndcg_score

# --- Crear DataFrame de Evaluación ---
print("Creando DataFrame de evaluación para 'Most Popular'...")
# Usaremos df_val como base, ya que contiene las interacciones reales a evaluar.
df_eval = df_val.copy()
df_eval = df_eval.rename(columns={'user': 'userID', 'item': 'itemID'})

# El 'score' para el modelo "Most Popular" es el rating promedio precalculado.
df_eval['score'] = df_eval['itemID'].map(item_avg_ratings_dict).fillna(global_avg_rating)
# 'label' indica si el ítem fue relevante (rating >= 7)
df_eval['label'] = (df_eval['rating'] >= 7).astype(int)
print("DataFrame de evaluación creado.")


# --- Funciones de métrica de ranking (puedes moverlas a una celda común si prefieres) ---
def precision_recall_at_k(group, k):
    group = group.sort_values('score', ascending=False)
    topk = group.head(k)
    hits = topk['label'].sum()
    total_relevant = group['label'].sum()
    precision = hits / k if k > 0 else 0
    recall = hits / total_relevant if total_relevant > 0 else 0
    return precision, recall

def ndcg_at_k(group, k):
    if group['label'].sum() == 0: return 0.0
    ranked_group = group.sort_values('score', ascending=False).head(k)
    if len(ranked_group) < 2: return 0.0
    true_relevance = np.asarray([ranked_group['label'].values])
    predicted_scores = np.asarray([ranked_group['score'].values])
    return ndcg_score(true_relevance, predicted_scores)


# --- Evaluación Individual Completa ---
K_values = [10]
individual_results = []
print("\nCalculando métricas individuales para Most Popular...")

grouped_users = df_eval.groupby('userID')

for k in K_values:
    metrics = grouped_users.apply(lambda x: precision_recall_at_k(x, k))
    avg_precision = np.mean([m[0] for m in metrics])
    avg_recall = np.mean([m[1] for m in metrics])
    avg_ndcg = grouped_users.apply(lambda x: ndcg_at_k(x, k)).mean()
    avg_novelty = grouped_users.apply(lambda x: novelty_at_k(x, k, item_popularity_prob)).mean()
    avg_diversity = grouped_users.apply(lambda x: diversity_at_k(x, k, df_mechanics)).mean()

    individual_results.append({
        'K': k,
        'Precision@K': avg_precision,
        'Recall@K': avg_recall,
        'nDCG@K': avg_ndcg,
        'Novelty@K': avg_novelty,
        'Diversity@K': avg_diversity
    })

individual_results_df = pd.DataFrame(individual_results)
print("\n--- Resultados de Evaluación Individual (Most Popular) ---")
print(individual_results_df)

Creando DataFrame de evaluación para 'Most Popular'...
DataFrame de evaluación creado.

Calculando métricas individuales para Most Popular...

--- Resultados de Evaluación Individual (Most Popular) ---
    K  Precision@K  Recall@K   nDCG@K  Novelty@K  Diversity@K
0  10     0.475039  0.840451  0.83743   10.62011     0.737274


Ahora haremos lo mismo pero para un grupo de 4 personas que han calificado ese item (haremos 4 personas por mas que haya juegos que es de máximo 2 o 10 etc por simplicidad). Lo que haremos es ver cómo se compara la recomendación (modelo most popular) versus su promedio de calificaciones que le dieron al item.

Para este nos basamos en lo hecho en random pa

In [15]:
def evaluar_mostpopular_topn_grupos(df_train, df_val, n=10):
    """
    Genera recomendaciones tipo Most Popular para grupos de 4 usuarios.
    Para cada grupo se calculan candidatos (items no vistos por NADIE del grupo)
    y se puntúan por rating promedio del item (popularidad).
    Devuelve: { (u1,u2,u3,u4): [(item, score), ...] }
    """
    # Popularidad por item (promedio de rating) y promedio global
    item_avg = df_train.groupby('item')['rating'].mean().to_dict()
    global_avg = df_train['rating'].mean()

    # Items disponibles y "vistos" por usuario en train
    all_items = df_train['item'].unique().tolist()
    user2seen = df_train.groupby('user')['item'].apply(set).to_dict()

    # Armar grupos secuenciales de 4 a partir de los usuarios en validación
    usuarios = df_val['user'].unique().tolist()
    grupos = [tuple(usuarios[i:i+4])
              for i in range(0, len(usuarios) - (len(usuarios) % 4), 4)]

    top_n_grupo = {}

    for grupo in grupos:
        # Unión de items vistos por cualquiera del grupo
        seen_union = set()
        for uid in grupo:
            seen_union |= user2seen.get(uid, set())

        # Candidatos: no vistos por nadie del grupo
        candidates = list(set(all_items) - seen_union)
        if not candidates:
            top_n_grupo[grupo] = []
            continue

        # Puntuar por popularidad (fallback al promedio global si no hay info)
        scores = [(iid, item_avg.get(iid, global_avg)) for iid in candidates]

        # Ordenar por score descendente y tomar top-n
        scores.sort(key=lambda x: x[1], reverse=True)
        top_n_grupo[grupo] = scores[:n]

    return top_n_grupo


def rmse_mae_from_topn_grupo(top_n_grupo, df_val):
    """
    Calcula RMSE/MAE comparando las predicciones del grupo (score del item)
    con los ratings reales en df_val para miembros del grupo.
    Nota: imita la lógica de 'random' tomando los ratings disponibles del grupo.
    """
    real, predicho = [], []

    total = sum(len(recs) for recs in top_n_grupo.values())
    i = 0

    for grupo, recs in top_n_grupo.items():
        for iid, pred in recs:
            # Tomamos TODOS los ratings disponibles en el grupo para ese item
            real_vals = df_val.loc[
                (df_val['user'].isin(grupo)) & (df_val['item'] == iid),
                'rating'
            ].values

            # Si hay varios miembros con rating para ese item, contamos todos
            for rv in real_vals:
                real.append(rv)
                predicho.append(pred)

            i += 1
            if total > 0 and (i % 10000 == 0 or i == total):
                progreso = (i / total) * 100
                print(f"Progreso: {i}/{total} ({progreso:.2f}%)")

    # Si no hubo pares comparables, devolvemos NaN
    if len(real) == 0:
        return float('nan'), float('nan')

    return sqrt(mean_squared_error(real, predicho)), mean_absolute_error(real, predicho)


# ====== Ejecución (igual al estilo de las últimas celdas de random) ======
start_time = time.time()
top_n_grupo = evaluar_mostpopular_topn_grupos(df_train, df_val, n=10)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución (generación top-n grupos): {elapsed_time:.2f} segundos")

#memoria = memory_usage(
#    (evaluar_mostpopular_topn_grupos, (df_train, df_val), {'n':10})
#)
#print("Memoria usada (MB):", max(memoria) - min(memoria))

# Métricas

#print("RMSE grupos (Most Popular):", rmse_grupo)
#print("MAE grupos (Most Popular):", mae_grupo)

Tiempo de ejecución (generación top-n grupos): 1219.12 segundos


In [16]:
print("\nCreando grupos sintéticos...")
user_counts = df_eval['userID'].value_counts()
valid_users = user_counts[user_counts >= 10].index.tolist()

np.random.seed(42)
num_groups = 1000
group_size = 4
groups = [np.random.choice(valid_users, group_size, replace=False) for _ in range(num_groups)]
print(f"Se crearon {len(groups)} grupos sintéticos de tamaño {group_size}.")

print("\nAgregando predicciones para cada grupo...")
all_group_recs = []
for group_id, user_ids in enumerate(groups):
    group_predictions = df_eval[df_eval['userID'].isin(user_ids)]
    item_scores_per_group = group_predictions.groupby('itemID').agg(
        # Para Most Popular, el score es el mismo para todos, por lo que avg=min=max.
        # Mantenemos las 3 estrategias por consistencia con otros modelos.
        avg_score=('score', 'mean'),
        min_score=('score', 'min'),
        max_score=('score', 'max'),
        # Ground Truth: ítem relevante si a TODOS les gustó
        group_label=('label', lambda x: 1 if all(x == 1) else 0)
    ).reset_index()
    item_scores_per_group['group_id'] = group_id
    all_group_recs.append(item_scores_per_group)

df_group_eval = pd.concat(all_group_recs, ignore_index=True)
print("Agregación completada.")

# --- Evaluación de Estrategias con Todas las Métricas ---
strategies = {
    'Average': 'avg_score',
    'Least Misery': 'min_score',
    'Most Pleasure': 'max_score'
}

group_results = []
K_values = [10]

for strategy_name, score_column in strategies.items():
    print(f"\nEvaluando estrategia (Most Popular): {strategy_name}...")
    df_strategy_eval = df_group_eval[['group_id', 'itemID', 'group_label']].copy()
    df_strategy_eval.rename(columns={'group_label': 'label'}, inplace=True)
    df_strategy_eval['score'] = df_group_eval[score_column]

    grouped_strategy = df_strategy_eval.groupby('group_id')

    for k in K_values:
        # Métricas existentes
        metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
        avg_precision = np.mean([m[0] for m in metrics])
        avg_recall = np.mean([m[1] for m in metrics])
        avg_ndcg = grouped_strategy.apply(lambda x: ndcg_at_k(x, k)).mean()

        # Nuevas métricas
        avg_novelty = grouped_strategy.apply(lambda x: novelty_at_k(x, k, item_popularity_prob)).mean()
        avg_diversity = grouped_strategy.apply(lambda x: diversity_at_k(x, k, df_mechanics)).mean()

        group_results.append({
            'Model': 'Most Popular',
            'Strategy': strategy_name,
            'K': k,
            'Precision@K': avg_precision,
            'Recall@K': avg_recall,
            'nDCG@K': avg_ndcg,
            'Novelty@K': avg_novelty,
            'Diversity@K': avg_diversity
        })

group_results_df = pd.DataFrame(group_results)

print("\n--- Resultados de Evaluación Grupal para Most Popular ---")
print(group_results_df)


Creando grupos sintéticos...
Se crearon 1000 grupos sintéticos de tamaño 4.

Agregando predicciones para cada grupo...
Agregación completada.

Evaluando estrategia (Most Popular): Average...

Evaluando estrategia (Most Popular): Least Misery...

Evaluando estrategia (Most Popular): Most Pleasure...

--- Resultados de Evaluación Grupal para Most Popular ---
          Model       Strategy   K  Precision@K  Recall@K    nDCG@K  \
0  Most Popular        Average  10       0.8999  0.145205  0.967531   
1  Most Popular   Least Misery  10       0.8999  0.145205  0.967531   
2  Most Popular  Most Pleasure  10       0.8999  0.145205  0.967531   

   Novelty@K  Diversity@K  
0  10.091293     0.805881  
1  10.091293     0.805881  
2  10.091293     0.805881  
